# 1 - Récupération des données

Les réglementations étant assez vastes, nous allons dans un premier temps nous concentrer sur les réglementations liées à la sécurité incendie.

Cette dernière est propre aux types d'ouvrages construits parmi lesquels nous retrouvons :
- les ERT ou établissement recevant des travailleurs
- les ERP ou établissement recevant du publics
- les batiments d'habitation
(etc) et tous ont leurs propres lois et décrets.

## 1-1 Scrapping des données du Code du travail (ERT)

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000006072050/LEGISCTA000018488606/#LEGISCTA000018532586"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    articles_data = []

    articles = soup.find_all('a', id=lambda x: x and x.startswith("linkLEGIARTI"))
    
    for article in articles:

        article_name = article.get_text(strip=True)
        
        mise_en_application = article.find_next('a', href=lambda x: x and '/loda/id/' in x)
        mise_en_application_text = mise_en_application.get_text(strip=True) if mise_en_application else 'Non disponible'
        
        content_article = article.find_next('div', class_='content')
        if content_article:
            content_p = content_article.find('p')
            if content_p:
                content = content_p.get_text(separator="\n", strip=True)
            else:
                content = 'Contenu non disponible'
        else:
            content = 'Contenu non disponible'

        articles_data.append({
            'Article': article_name,
            'Mise en application': mise_en_application_text,
            'Contenu': content
        })

    df = pd.DataFrame(articles_data)
    display(df)

else:
    print("Erreur lors du chargement de la page :", response.status_code)


,Article,Mise en application,Contenu
0,Article R4211-1,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Les dispositions du présent titre déterminent,..."
1,Article R4211-2,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Pour l'application du présent titre, on entend..."
2,Article R4211-3,Modifié par Décret n°2011-1461\n du 7 novembre...,Le maître d'ouvrage élabore et transmet aux ut...
3,Article R4211-4,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...
4,Article R4211-5,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...
...,...,...,...
294,Article R4228-37,Création Décret n°2008-244 du 7 mars 2008 - ar...,Les dispositions relatives à l'hébergement des...
295,Article R4231-1,Création DÉCRET n°2015-364 du 30 mars 2015 - a...,Contenu non disponible
296,Article R4231-2,Création DÉCRET n°2015-364 du 30 mars 2015 - a...,"Dès réception de l'injonction, l'employeur inf..."
297,Article R4231-3,Création DÉCRET n°2015-364 du 30 mars 2015 - a...,En cas d'absence de régularisation effective d...


## 1-2 Scrapping des données des Etablissements recevant du public (ERP)

## 1-3 Scrapping des données du Code de la construction et de l’habitation (CCH)